In [2]:
import pandas as pd 
import requests as req

In [85]:
regist = pd.read_csv('./regs_1k.csv')
visits = pd.read_csv('./visits_1k.csv')

print(f'Registration\n {regist.sort_values('date').tail()}')

print(f'\nVisits\n {visits.sort_values('date').head()}')

Registration
                     date  user_id                      email platform  \
947  2023-03-05T21:11:40  8509874         zsmith@example.org      web   
969  2023-03-05T21:50:21  1557199    watsonsarah@example.org      web   
970  2023-03-05T21:52:46  8833853  robinsonkatie@example.net      web   
965  2023-03-05T22:00:42  6650724    andrewparks@example.com      web   
973  2023-03-05T22:04:01   393104   kelseynewman@example.com      web   

    registration_type  
947             apple  
969            google  
970            yandex  
965            google  
973             apple  

Visits
                                      uuid platform  \
264  d72d0452-c34d-4a29-9e7d-ad1cee0256b8      web   
699  0e63cfdc-84dc-4bf9-a646-aef87212761d      web   
790  a1772ff8-4cfa-47fb-b192-e9b8347fa7c0      web   
346  7827eafe-c8cf-4472-9b29-07132085dc12      web   
285  bcd5f9f0-dd3f-43e7-87a1-1010cb626433      web   

                                            user_agent               

In [ ]:
START = '2023-03-01'
END = '2023-09-01'
visitors_req = req.get('https://data-charts-api.hexlet.app/visits?begin={START}&end={END}')
print(visitors_req)
# visitors = pd.DataFrame(visitors_req.json())
regs_req = req.get('https://data-charts-api.hexlet.app/registrations?begin={START}&end={END}')
# regs = pd.DataFrame(regs_req.json())

# resp = requests.get('http://localhost:9000/visits')
# data = pd.DataFrame(resp.json())

<Response [500]>


In [88]:
start_date = '2023-03-01'
end_date = '2023-09-01'

regist = regist.sort_values('date')

visits = visits.sort_values('date')
regist['date'] = pd.to_datetime(regist['date'])
regist['date'] = regist['date'].dt.strftime('%Y-%m-%d')
filtered_regist = regist[(regist['date'] >= start_date) & (regist['date'] <= end_date)]

visits['date'] = pd.to_datetime(visits['date'])
visits['date'] = visits['date'].dt.strftime('%Y-%m-%d')
filtered_visits = visits[(visits['date'] >= start_date) & (visits['date'] <= end_date)]

# print(f'Registration\n {filtered_regist.tail()}')
# print(f'\nVisits\n {filtered_visits.head()}')


In [ ]:
# regist_2 = regist.drop_duplicates(subset='user_id', keep='last')
# regist_2['date'] = pd.to_datetime(regist_2['date'])
# filtered_regist_2 = regist_2[(regist_2['date'] >= start_date) & (regist_2['date'] <= end_date)]

# visits_2 = regist.drop_duplicates(subset='user_id', keep='last')
# visits_2['date'] = pd.to_datetime(visits_2['date'])
# filtered_visits_2 = visits_2[(visits_2['date'] >= start_date) & (visits_2['date'] <= end_date)]

# print(f'Registration\n {filtered_regist_2.head()}')
# print(f'\nVisits\n {filtered_visits_2.head()}')

In [89]:
grouped_regist = filtered_regist.groupby('platform')['user_id'].count().reset_index(name='regist_count')
grouped_regist_date = filtered_regist.groupby('date')['user_id'].count().reset_index(name='regist_count')
# grouped_regist.to_json('./result.json')

grouped_visits = filtered_visits.groupby('platform')['uuid'].count().reset_index(name='visits_count')
grouped_visits_date = filtered_visits.groupby('date')['uuid'].count().reset_index(name='visits_count')
# grouped_visits.to_json('./result_2.json')
print(grouped_regist_date)
print(grouped_regist)
print(grouped_visits)
print(grouped_visits_date)




         date  regist_count
0  2023-03-01            60
1  2023-03-02           320
2  2023-03-03           279
3  2023-03-04           288
4  2023-03-05            53
  platform  regist_count
0  android           517
1      ios           218
2      web           265
  platform  visits_count
0  android            27
1      ios            19
2      web           954
         date  visits_count
0  2023-03-01           510
1  2023-03-02           149
2  2023-03-03            98
3  2023-03-04           105
4  2023-03-05            94
5  2023-03-06            40
6  2023-03-07             4
